In [62]:
# button customisation.py / should be called within the sound thread
# CUSTOMISATION

from os import walk
import pyttsx3
engine = pyttsx3.init()

def change_voice_property(index):
    f = open(CACHE_JSON_FILE)
    data = json.load(f)

    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[index].id)
    
    f = []
    for (dirpath, dirnames, filenames) in walk(MSG_CACHE_PATH):
        f.extend(filenames)
        break
        
#     How to to know what was said..... create a directory of what is said... (chnage the code)
    for elt in f:
        print(elt)
        save_msg_to_cache(data[elt],elt)
    
    f.close()
    
# dummy commands to be added
def update_type_of_beep():
    return

def update_num_beeps():
    return
    

# logging should moniter the success rate to the file 
def select_an_option():
    res = listen()
    if "quit" in res:
        add_log("VOICE COMMAND,FAIL,"+res)
        play_msg_cache('Quitting')
        return None
    else:
#         could cache this possibly...??
        engine.say('Is '+ res + " correct?")
        engine.runAndWait()

        rez = listen()
        if "yes" in rez:
            add_log("VOICE COMMAND,Success,"+res)
            play_msg_cache('Great_Updating_Now')
            return res
        elif "quit" in rez or "no" in rez :
            add_log("VOICE COMMAND,FAIL,"+res)
            play_msg_cache('Quitting')
            return None
        else:
            add_log("VOICE COMMAND,FAIL,"+res)
            play_msg_cache('say_again')
            select_an_option()
        

def cycle_voices():
    voices = engine.getProperty('voices')

    for y in range(5):
        engine.setProperty('voice', voices[y].id)
        engine.say('Voice '+str(y))
        engine.runAndWait()
        
    play_msg_cache('voice_number_start')
        
#     call the selection method
    ind = select_an_option
    if ind is not None:
        update_cache_commands(ind)

    
def cycle_beeps():
#    need to change once in main program
    mypath = '/Users/euanchalmers/Desktop/SDP/yolov5/fnd/beeps'

    f = []
    for (dirpath, dirnames, filenames) in walk(mypath):
        f.extend(filenames)
        break

    print(f)
    engine = pyttsx3.init()

    for idx,each in enumerate(f):
        if '.wav' in each:
            engine.say('Beep '+str(idx+1))
            engine.runAndWait()

            file_name = mypath + '/' + each

            [y, sr] = librosa.load(file_name, sr=48000)
            duration = librosa.get_duration(filename=file_name) * 1000  # value in ms

            sd.play(y, sr)
            sd.sleep(int(duration))
            sd.stop()
            
    play_msg_cache('beep_type_start')
    
    ind = select_an_option()
    if ind is not None:
        update_type_of_beep(ind)
            
def customise_number_beeps():
    play_msg_cache('customise_beeps_start')
#     how to speed up the wait here
    
    ind = select_an_option()
    if ind is not None:
        update_num_beeps(ind)


In [59]:
#-------------------------------------------------
# VoiceToText.py should also happen in the sound thread
# ------------------------------------------------

import pyttsx3
import librosa
import sounddevice as sd
import os
import json

MSG_CACHE_PATH = '/Users/euanchalmers/Desktop/full_sdp_3-main/Main/fnd/recorded_msg'
# need to add
CACHE_JSON_FILE = ''

def save_msg_to_cache(input_text, file_name):
    if 'wav' not in file_name:
        file_name = file_name + '.wav'

    engine = pyttsx3.init()
    engine.save_to_file(input_text, str(file_name))
    engine.runAndWait()
    engine.stop()
    print('written to...',file_name)


def play_msg_cache(file_name):
    if '.wav' not in file_name:
        file_name = file_name+'.wav'

    [y, sr] = librosa.load(file_name, sr=48000)
    duration = librosa.get_duration(filename=file_name) * 1000  # value in ms

    sd.play(y, sr)
    sd.sleep(int(duration))
    sd.stop()

    
#     ADD HERE -----------------------------------------------------------------------------------
import speech_recognition as sr

# on button press start listening and return command
def listen():

    r  = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, duration=0.2)
        print("Speak now")
#         try:
        audio = r.listen(source) #Starts Listening
#         except:
#             print("Sorry, could not hear you!")
#             return None
    try:
        text = r.recognize_google(audio) #Recognizes audio in English 
        return (text)
    
    except: #When there is no notable speech
        print("Sorry, couldn't hear you!")
        return None
    
# move to buttons or make a constants file
COMMANDS = ["pause","resume","stop","read out full","customise beep sound","customise voice","customise number of beeps"]
COMMANDS_BUTTONS = {"pause":"p","resume":"r","stop":"s","read out full": "m", "customise beep sound":"bs",
                    "customise voice":"cv", "customise number of beeps":"cnb"}

def parse(voice_command):
    print(voice_command)
    for o in COMMANDS:
        if o in voice_command:
            print("Command said is ... ",o)
            add_log("VOICE COMMAND,SUCCESS,"+str(voice_command))
            return COMMANDS_BUTTONS[o]
    
#     else condition
    add_log("VOICE COMMAND,FAIL,"+str(voice_command))
    play_msg_cache('default')
    return None

# call when the voice command is called
def voice_wrapper():
    res = listen()

    if res is None:
        play_msg_cache('default')
        return None
    else:
        return parse(res)

#     ----------------------------------------------------------------------------------


In [3]:
# testing diffrent mic sources/calibration??
# calibration of audio sources

import speech_recognition as sr
for index, name in enumerate(sr.Microphone.list_microphone_names()):
    print("Microphone with name \"{1}\" found for `Microphone(device_index={0})`".format(index, name))
    
import pyaudio

p = pyaudio.PyAudio()
try:
    print(p.get_default_input_device_info())
except:
    print("No mics availiable")

Microphone with name "Euan’s Beats Studio³" found for `Microphone(device_index=0)`
Microphone with name "Euan’s Beats Studio³" found for `Microphone(device_index=1)`
Microphone with name "MacBook Pro Microphone" found for `Microphone(device_index=2)`
Microphone with name "MacBook Pro Speakers" found for `Microphone(device_index=3)`
Microphone with name "Microsoft Teams Audio" found for `Microphone(device_index=4)`
{'index': 0, 'structVersion': 2, 'name': 'Euan’s Beats Studio³', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.014, 'defaultLowOutputLatency': 0.01, 'defaultHighInputLatency': 0.042, 'defaultHighOutputLatency': 0.1, 'defaultSampleRate': 16000.0}


In [63]:
# -------------------------------------------------------
# Deminstration of the voice to text
# --------------------------------------------------------
from IPython.display import display
import ipywidgets as widgets

# COMBINE THESE INTO ONE
COMMANDS_BUTTONS = {"pause":"p","resume":"r","stop":"s","read out full": "m", "customise beep sound":"bs",
                    "customise voice":"cv", "customise number of beeps":"cnb"}

# expand to all commands
BUTTONS_TO_METHODS = {"bs":cycle_voices ,"cv":cycle_voices,"cnb":customise_number_beeps}

def clicked(arg):
    arg = str(arg)
    itm = arg.split("description='")
    itm = itm[1]
    itm = itm.split("\'")
    itm = itm[0]
    
    if "Example Voice To Text" in itm:
        voice_wrapper()
    else:

        itm = COMMANDS_BUTTONS[itm]
        action = BUTTONS_TO_METHODS[itm]
        print(action)
        action()
        
        

button_download = widgets.Button(description = 'Example Voice To Text')
button_download.on_click(clicked)

button_two = widgets.Button(description = 'customise beep sound"')   
button_two.on_click(customise_number_beeps)

button_three = widgets.Button(description = 'customise voice')   
button_three.on_click(cycle_voices)

button_four = widgets.Button(description = 'customise number of beeps')   
button_four.on_click(customise_number_beeps)

 
display(button_download,button_two,button_three,button_four)

Button(description='Example Voice To Text', style=ButtonStyle())

Button(description='customise beep sound"', style=ButtonStyle())

Button(description='customise voice', style=ButtonStyle())

Button(description='customise number of beeps', style=ButtonStyle())

In [ ]:
# to log the files correctly
save_logs_to_file()